In [ ]:
import pandas as pd
import tkinter as tk
import warnings
import os
from tkinter import messagebox
from datetime import datetime
import calendar

warnings.filterwarnings('ignore')

def obtener_fecha(mes_anterior=False):
    hoy = datetime.now()
    
    if mes_anterior:
        if hoy.month == 1:
            mes = 12
            año = hoy.year - 1
        else:
            mes = hoy.month - 1
            año = hoy.year
    else:
        mes = hoy.month
        año = hoy.year
    
    mes_nombre = calendar.month_abbr[mes].upper()[:3] # ENE
    mes_año = f"{mes_nombre}{str(año)[2:]}" # ENE24
    fecha = f"{año}{str(mes).zfill(2)}" # 202401
    
    return mes_año, fecha

mes_año, fecha = obtener_fecha()

base_pagos_efect_path = f'bases/efectividades/{fecha}/BasePagos_Efectividades_20241031_Contrato.xlsx'
base_propuesta_castigo_path = f'bases/efectividades/{fecha}/Propuesta de Castigo_Perímetro 2024 - 10.xlsx'
base_asignacion_path = f'bases/asignacion/{fecha}/base_asignacion_NOV24_.xlsx'
base_castigo_path = f'bases/efectividades/{fecha}/base_castigo_{mes_año}.xlsx'

base_propuesta_castigo_path = os.path.abspath(base_propuesta_castigo_path)
base_asignacion_path = os.path.abspath(base_asignacion_path)
base_castigo_path = os.path.abspath(base_castigo_path)

print(base_pagos_efect_path)
print(base_propuesta_castigo_path)
print(base_asignacion_path)
print(base_castigo_path)

In [ ]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Cargar efectividades?', icon='warning')
if result == 'yes':
    df_pagos = pd.read_excel(base_pagos_efect_path, sheet_name='BD')

root.destroy()

In [ ]:
df_pagos_test = df_pagos.copy()

df_pagos_test['contrato'] = df_pagos_test['contrato'].apply(lambda x: str(int(x)).zfill(20) if pd.notna(x) else x)
df_pagos_test['contrato'] = df_pagos_test['contrato'].apply(lambda x: x[:8] + x[-10:] if pd.notna(x) else x)

cols_req = ['fecha', 'periodo', 'dia', 'Dia_Util', 'codcent', 'clave', 'Agencia', 'CodAgenciaDist', 'foco', 
    'contrato', 'META', 'flag_exclusion_findemes', 'oficina', 'MaxDA', 'NUM', 'CapitalSoles', 'PagoTotalSoles', 
    'PagoTotalDistribuida', 'PagoEfectTotalSoles', 'PagoEfectTotalSolesAgencia', 'PagoEfectTotalSolesAgenciaCont']

df_pagos_test = df_pagos_test[cols_req]
df_pagos_test.shape

In [ ]:
df_bc = pd.read_excel(base_propuesta_castigo_path, sheet_name='BC')
df_exj = pd.read_excel(base_propuesta_castigo_path, sheet_name='EXJ')
df_concat = pd.concat([df_bc, df_exj], axis=0)

df_concat['CONTRATO'] = df_concat['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_concat['CODCENTRAL'] = df_concat['CODCENTRAL'].astype(str)
df_concat.shape

In [ ]:
df_merged = pd.merge(df_pagos_test, df_concat, left_on='contrato', right_on='CONTRATO', how='left')
df_merged.drop(columns=['CONTRATO', 'INHIBIR', 'MOTIVO'], inplace=True)
df_merged['FLAG_CASTIGO'] = df_merged['CODCENTRAL'].apply(lambda x: 1 if pd.notna(x) else 0)
df_merged.shape

In [ ]:
df_merged['FLAG_CASTIGO'].value_counts()

In [ ]:
df_base_castigo = df_merged[df_merged['FLAG_CASTIGO']==1]
df_base_castigo.columns = clean_columns(df_base_castigo.columns)
df_base_castigo.shape

In [ ]:
df_asignacion = pd.read_excel(base_asignacion_path)
df_asignacion['CONTRATO'] = df_asignacion['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_asignacion.shape

In [ ]:
df_cartera_final = pd.merge(df_base_castigo, df_asignacion, on='CONTRATO', how='left')
df_cartera_final.rename(columns={'AGENCIA_x': 'AGENCIA', 'CLAVE': 'CARTERA', 'CARTERA': 'CARTERA_ASIGNACION'}, inplace=True)

cols_final = ['FECHA', 'PERIODO', 'DIA', 'DIA_UTIL', 'CODCENT', 'CARTERA', 'CARTERA_ASIGNACION', 'AGENCIA',
    'CODAGENCIADIST', 'FOCO', 'CONTRATO', 'META', 'FLAG_EXCLUSION_FINDEMES',
    'OFICINA', 'MAXDA', 'NUM', 'CAPITALSOLES', 'PAGOTOTALSOLES',
    'PAGOTOTALDISTRIBUIDA', 'PAGOEFECTTOTALSOLES',
    'PAGOEFECTTOTALSOLESAGENCIA', 'PAGOEFECTTOTALSOLESAGENCIACONT',
    'CODCENTRAL', 'CLIENTE', 'FLAG_CASTIGO']

df_cartera_final = df_cartera_final[cols_final]
df_cartera_final.shape

In [ ]:
df_cartera_final.to_excel(base_castigo_path, index=False)